# Datenbeschreibung


Im Rahmen unseres Projekts für Big Data haben sich die Teammitglieder Colin Sturm, Leonard Debinski, Daniel Gern, Tim Paschke, Yusuf Esentürk und Carlo Koppanyi zusammengeschlossen. Unser zentrales Forschungsthema ist die Kundensegmentierung und die Personalisierung von Marketingmaßnahmen. Um eine praxisnahe Herangehensweise zu gewährleisten, haben wir auf unserer Miro-Plattform drei detaillierte Personas entwickelt. Diese Personas reflektieren unterschiedliche Kundenprofile mit spezifischen Problemen und Bedürfnissen in Bezug auf unser Forschungsthema. Um diese Herausforderungen effektiv anzugehen, haben wir uns für die Analyse eines passenden Datensatzes entschieden, der darauf ausgerichtet ist, die Problematiken und Wünsche der Personas gezielt zu adressieren.
Bei der Auswahl eines geeigneten Datensatzes für unser Projekt legen wir großen Wert darauf, hauptsächlich quantitative Merkmale und nur wenige qualitative Merkmale zu berücksichtigen. Nach sorgfältiger Prüfung verschiedener Optionen kamen wir zu dem Schluss, dass der "Datensatz Online Shop Customer Sales Data" am besten unseren Anforderungen entspricht. Der Datensatz beinhaltet verschiedene Merkmale zu Kunden eines Online-Shops. Das Merkmal "Customer_id" dient als eindeutige Identifikation für jeden Kunden. Das Alter des Kunden wird durch "Age" wiedergegeben. Das Geschlecht des Kunden ist unter "Gender" kategorisiert, wobei 0 für männlich und 1 für weiblich steht. "Revenue_Total" zeigt den Gesamtumsatz an, den ein Kunde bisher generiert hat, während "N_Purchases" die Anzahl seiner Käufe bis zum aktuellen Datum anzeigt. Das Datum des letzten Kaufs wird im Format dd.mm.yy unter "Purchase_DATE" erfasst und der Wert dieses letzten Kaufs in Euro wird unter "Purchase_VALUE" angezeigt. Das Merkmal "Pay_Method" gibt an, welche Zahlungsmethode der Kunde verwendet hat, wobei 0 für digitale Geldbörsen, 1 für Karten, 2 für PayPal und 3 für andere Zahlungsmethoden steht. "Time_Spent" zeigt die in Sekunden verbrachte Zeit des Kunden auf der Webseite. Der von dem Kunden verwendete Browser wird durch "Browser" kategorisiert, wobei 0 für Chrome, 1 für Safari, 2 für Edge und 3 für andere Browser steht. Zusätzlich zeigt "Newsletter", ob ein Kunde für den Newsletter angemeldet ist oder nicht, wobei 0 bedeutet, dass er nicht abonniert ist und 1, dass er abonniert ist. Schließlich gibt "Voucher" an, ob ein Kunde einen Gutschein verwendet hat, wobei 0 für nicht verwendet und 1 für verwendet steht. Zu erkennen sind sechs quantitative Merkmale und vier qualitative Merkmale. 


In [10]:
#Bibliothekten
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor

# Datenvorbereitung

Es handelt sich bei der Datenvorbereitung um einen systematischen Prozess, die Datenqualität zu verbessern und die Genauigkeit der zukünftigen Analysen zu erhöhen.
Es gibt verschiedene Methoden, um einen Datensatz auf eine bevorstehende Analyse vorzubereiten.

 
 Zunächst werden generelle Informationen zum Datensatz angezeigt. Diese sind wichtig, da am Ende des Bereinigungsprozeses geprüft werden muss, ob sich die quantitiven Informationen wie Zeilenanzahl geändert haben. Diese Informationen sind wichtig, da sie bei der Modellbildung verwendet werden.

In [11]:

# Daten einlesen
df = pd.read_csv('Online Shop Customer Sales Data.csv')

# ausgeben der quantitativen Informationen zum Datensatz
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65796 entries, 0 to 65795
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Customer_id     65796 non-null  int64  
 1   Age             65796 non-null  int64  
 2   Gender          65796 non-null  int64  
 3   Revenue_Total   65796 non-null  float64
 4   N_Purchases     65796 non-null  int64  
 5   Purchase_DATE   65796 non-null  object 
 6   Purchase_VALUE  65796 non-null  float64
 7   Pay_Method      65796 non-null  int64  
 8   Time_Spent      65796 non-null  int64  
 9   Browser         65796 non-null  int64  
 10  Newsletter      65796 non-null  int64  
 11  Voucher         65796 non-null  int64  
dtypes: float64(2), int64(9), object(1)
memory usage: 6.0+ MB
None


Nun gilt es zu prüfen, ob die Daten im richtigen Format vorliegen. Hierzu betrachten wir die Datentypen.
Dies ist wichtig, da verschiedene Datentypen unterschiedliche statistische Analysen erfordern.

astype(): Die Funktion ist ein nützliches Werkzeug in Pandas, das es ermöglicht, den Datentyp einer Spalte zu ändern.

Zunächst müssen jedoch die einzelnen Datentypen der Spalten identifiziert werden. Hierzu wird folgende Funktion verwendet: 
print(df.info())

Es ergibt, dass alle Datentypen korrekt identifiziert wurden. Eine Ausnahme bildet die Spalte „Purchase_DATE“. Um die Werte korrekt in ein gültiges Datumsformat zu packen Bedarf es 2 Schritte: 

1.	Prüfen, ob ein unmögliches Datum enthalten ist
2.	Die Zeichenkette in ein einheitliches Format (dd.mm.yy) konvertieren
Hierzu wird die strptime() Funktion aus dem datetime Modul in Python verwendet. Diese Funktion versucht, eine Zeichenkette in ein Datum zu konvertieren. Wenn die Zeichenkette kein gültiges Datum ist, wird ein ValueError ausgelöst.

Ein Datum gilt als gültig, wenn es folgende Kriterien erfüllt:

Format: Ein gültiges Datum sollte ein bestimmtes Format haben. In der Regel wird das Datum im Format 'Tag.Monat.Jahr' angegeben, wobei Tag, Monat und Jahr durch Punkte getrennt sind. 
Werte: Ein gültiges Datum sollte gültige Werte für Tag, Monat und Jahr haben. Beispielsweise sollte der Tag zwischen 1 und 31 liegen, der Monat zwischen 1 und 12 und das Jahr ein gültiges Jahr sein (z.B. 2023). Wenn eines dieser Kriterien nicht erfüllt ist, ist das Datum nicht gültig.
Konsistenz: Ein gültiges Datum sollte konsistent sein. Beispielsweise sollte ein Datum im Format '30.02.2023' nicht gültig sein, da der Februar nicht 30 Tage hat. Wenn das Datum nicht konsistent ist, ist es nicht gültig.


In [12]:
# Überprüfen der Datentypen
print(df.info())

# Konvertieren der Datentypen (Purchase_DATE)
from datetime import datetime

def convert_to_date(date_str):
  try:
      # Versuch, das Datum zu konvertieren
      return pd.to_datetime(date_str, format="%d.%m.%y")
  except ValueError:
      # Wenn ein ValueError ausgelöst wird, ist das Datum nicht gültig
      print("hi")
      return pd.NaT

# Konvertieren des Datums in jeder Zeile
df['Purchase_DATE'] = df['Purchase_DATE'].apply(convert_to_date)


# Erneutes Überprüfen der Datentypen
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65796 entries, 0 to 65795
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Customer_id     65796 non-null  int64  
 1   Age             65796 non-null  int64  
 2   Gender          65796 non-null  int64  
 3   Revenue_Total   65796 non-null  float64
 4   N_Purchases     65796 non-null  int64  
 5   Purchase_DATE   65796 non-null  object 
 6   Purchase_VALUE  65796 non-null  float64
 7   Pay_Method      65796 non-null  int64  
 8   Time_Spent      65796 non-null  int64  
 9   Browser         65796 non-null  int64  
 10  Newsletter      65796 non-null  int64  
 11  Voucher         65796 non-null  int64  
dtypes: float64(2), int64(9), object(1)
memory usage: 6.0+ MB
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65796 entries, 0 to 65795
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Customer_id     65796 non-null  int64         
 1   Age             65796 non-null  int64         
 2   Gender          65796 non-null  int64         
 3   Revenue_Total   65796 non-null  float64       
 4   N_Purchases     65796 non-null  int64         
 5   Purchase_DATE   65796 non-null  datetime64[ns]
 6   Purchase_VALUE  65796 non-null  float64       
 7   Pay_Method      65796 non-null  int64         
 8   Time_Spent      65796 non-null  int64         
 9   Browser         65796 non-null  int64         
 10  Newsletter      65796 non-null  int64         
 11  Voucher         65796 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(9)
memory usage: 6.0 MB
None


Anschließend wird der gesamte Datensatz überflogen, um ein grobes Verständnis für die Daten zu gewinnen. Hierfür wird der Datensatz angezeigt.

Danach gilt es zu prüfen, ob es Null Werte oder Werte gibt, die auf einen Null- Wert hinweisen. Strings, welche die Zeichenkette "N/A" enthalten stehen oftmals für "Not Available", also Null- Werte.
Wenn es mindestens einen "N/A" Wert gibt, wird der gesamte Datensatz bereinigt und durch geeignete statistische Mittel ersetzt. Die statitischen Mittel unterscheiden sich je nach Datentyp und Kontext der Variable.

Trivialerweise ist Letzteres nicht von Nöten. Der Grund hierfür liegt darin, dass alle Spalten bereits die korrekten Datentypen besitzen. In diesem Fall haben wir keine Spalte mit Stringwerten, weshalb der Schritt übersprungen werden kann. 
Dennoch ist dies ein wichtiger Schritt, falls Daten mit Stringwerten im zukünftigen Verlauf des Projekts verwendet werden könnten. Dann muss dieser Schritt auf diese Werte angewendet werden.


In [13]:
# Anzeigen des Datensatzes
df

# Überprüfen Sie, ob Nullwerte im Datensatz vorhanden sind
print(df.isnull().sum())

# Überprüfen Sie, ob "N/A" im Datensatz vorhanden ist
# print(df.apply(lambda x: x.str.contains('N/A', na=False).any()))

Customer_id       0
Age               0
Gender            0
Revenue_Total     0
N_Purchases       0
Purchase_DATE     0
Purchase_VALUE    0
Pay_Method        0
Time_Spent        0
Browser           0
Newsletter        0
Voucher           0
dtype: int64


Des Weiteren gilt es zu prüfen, ob sich in den Datensatz Outlier eingeschlichen haben. Outlier, auch Ausreißer genannt, sind Datenpunkte in einem Datensatz, die signifikant von den anderen Werten abweichen. Sie sind entweder extrem hoch oder extrem niedrig im Vergleich zu den umliegenden Werten. Diese extreme Abweichung kann das Gesamtergebnis statistischer Analysen beeinflussen und möglicherweise auf Fehler in den statistischen Verfahren hinweisen.

df.quantile(0.25) und df.quantile(0.75): Diese Funktionen berechnen die ersten und dritten Quartile des Datensatzes. Quartile sind nützliche statistische Maße, die zur Erkennung von Outliers verwendet werden können.

df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]: Diese Funktion erkennt und behandelt Outlier im Datensatz. Sie entfernt alle Zeilen, die als Outlier erkannt werden.



Dieser Code wird von uns jedoch nicht verwendet. Denn es hat sich ergebeb, dass über 12000 Zeilen als Ausreißer identifiziert werden, was sehr unüblich ist.

In [ ]:
# Purchase_DATE Spalte in einen numerischen Datentyp umwandeln, um Ausreißer identifizieren zu können
# df['Purchase_DATE'] = df['Purchase_DATE'].apply(lambda x: x.timestamp())


# Erkennen von Outlier/ Ausreißer
# Q1 = df.quantile(0.25)
# Q3 = df.quantile(0.75)
# IQR = Q3 - Q1

# Behandeln von Outlier
# df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

Zuletzt betrachten wir die Minima und Maxima der verschiedenen Werte. Hier ist klar zu erkennen, dass keine Ausreißer vorhanden sind.

In [14]:
min_values = df.min()
max_values = df.max()
print("Min")
print(min_values)
print("Max")
print(max_values)

Min
Customer_id                    504308
Age                                16
Gender                              0
Revenue_Total                     0.5
N_Purchases                         1
Purchase_DATE     2021-01-01 00:00:00
Purchase_VALUE                  0.005
Pay_Method                          0
Time_Spent                        120
Browser                             0
Newsletter                          0
Voucher                             0
dtype: object
Max
Customer_id                    570103
Age                                63
Gender                              1
Revenue_Total                    59.9
N_Purchases                         7
Purchase_DATE     2022-01-01 00:00:00
Purchase_VALUE                   59.9
Pay_Method                          3
Time_Spent                       1080
Browser                             3
Newsletter                          1
Voucher                             1
dtype: object


**Standardisieren**
 
Algorithmen gehen in der Regel davon aus, dass Variablen auf einer gleichen Skala sind. Dies entspricht jedoch nicht immer der Realität. Deshalb ist es notwendig die für die Analysen benötigten Variablen zu skalieren.
Somit kann vermieden werden, das bestimmte Variablen einen überproportionalen Einfluss auf unser Modell haben.

In diesem Fall verwenden wir die Standardisierung. Bei dieser Methode werden die Daten so skaliert, dass sie einen Durchschnittswert von 0 und einen Standardabweichung von 1 haben.

Um zu vermeiden, dass Informationen aus den Testdaten in die Trainigsphase "geleaked" werden, sollten nur die Trainingsdaten skaliert werden. Das bedeutet, dass der Standardisierungsprozess von den Teammitgliedern aufgegriffen werden muss, da die Verteilung von Trainings- und Testdaten zufällig erfolgt.

Es bleibt zu erwähnen, dass aus zeittechnischen Gründen und der parallelen Zusammenarbeit zu Folge nicht die Möglochkeit bestand die Daten für die einzelnen Modelle zugeschnitten zu bereinigen.

Deshalb wird unten der allgemeine Ansatz beschrieben, den es für jedes unterschiedliche Modell zu verfeinern und zu implementieren gilt.

In [28]:
# Erstellen eines StandardScaler-Objekts
scaler = StandardScaler()

# Definieren von Merkmalen und Zielvariablen
features = ['Age', 'Revenue_Total', 'N_Purchases']
merkmale = df[features]

# Die Spalte, die predicted werden soll
merkmale_y = df['Time_Spent']  

# Instanziieren des StandardScaler-Objekts
scaler = StandardScaler()

# Aufteilen des Datensatzes in Trainings- und Testdaten
merkmale_train, merkmale_test, y_train, y_test = train_test_split(merkmale, merkmale_y, test_size=0.3, random_state=42)

# Fitten und transformieren des StandardScaler-Objekts mit den Trainingsdaten 
merkmale_train_scaled = scaler.fit_transform(merkmale_train)

# Transformieren der Testdaten
merkmale_test_scaled = scaler.transform(merkmale_test)

# Regression

## Ziel
Schätzen, wie viel der Kunde insgesamt ausgibt. Daraus lässt sich ein Potenzial bei Kunden berechnen, bei denen die momentanen Ausgaben bekannt sind.

## Schritte

1. Datensatz laden und Imports
2. Datenauswahl
3. Aufteilen in Trainings-, Test- und Validierungsdaten (keine Crossvalidation, weil genug Daten vorhanden sind)
4. Modelle mit dem Trainingsdatensatz berechnen 
5. Modellauswahl mittels der Validierungsdaten. Metrik: MQA
6. Berechnung des Testfehlers. Metrik: MQA
7. Anpassung des Modells
8. Tree
9. Auswahl des Modells
10. Fazit


1. **Datensatz laden und Imports**

In [15]:
data_reg = df

df_reg = pd.DataFrame(data_reg)
print(df_reg.head())
durchschnitt_revenue = df_reg['Revenue_Total'].mean()
print(durchschnitt_revenue)

   Customer_id  Age  Gender  Revenue_Total  N_Purchases Purchase_DATE  \
0       504308   53       0           45.3            2    2021-06-22   
1       504309   18       1           36.2            3    2021-12-10   
2       504310   52       1           10.6            1    2021-03-14   
3       504311   29       0           54.1            5    2021-10-25   
4       504312   21       1           56.9            1    2021-09-14   

   Purchase_VALUE  Pay_Method  Time_Spent  Browser  Newsletter  Voucher  
0          24.915           1         885        0           0        0  
1           2.896           2         656        0           0        1  
2          10.600           0         761        0           1        0  
3          43.280           1         906        0           1        0  
4          56.900           1         605        0           1        0  
27.732935132834818


2. **Datenauswahl**

Zuerst werden aus den qualitativen Daten, mit mehr als 2 Variablen, Dummyvariablen erzeugt. Trivialerweise sind Customer_id und Purchase_DATE davon ausgeschlossen.
Danach versuchen wir die Inputmerkmale durch eine Korrelationsanalyse zu reduzieren. Ab einer Korrelation größer als 0,5 werden sich die Merkmale näher angeschaut.

### Irrelevante Daten für die Korrelation:
- Customer_id      -->> kein Merkmal für die Regression (Schlüssel zum User)
- Purchase_DATE    -->> Qualitative Daten mit vielen Ausprägungen

### Korrelationen:
Browser_0 ~ Browser_1 -> -0.665166
- Browser 0 und Browser 1 werden zum Interpretieren behalten.
Jedoch wird die Variable Purchase_VALUE herausgenommen. Grund hierfür ist, dass die Variable den Wert des letzten Kaufes darstellt. Bei First-Time-Shoppern müsste dann dieser mit Revenue_Total übereinstimmen. Dies als auch die Varianz würden in diesem Beispiel das Ergebnis verfälschen.



In [25]:
df_dummy = pd.get_dummies(df_reg, columns=['Pay_Method', 'Browser'], prefix=['Pay_Method', 'Browser'])

df_dummy = df_dummy.drop(columns=['Purchase_DATE', 'Customer_id','Revenue_Total'])

print(df_dummy.corr())


df_dummy = df_dummy.drop(columns=['Purchase_VALUE'])
df_dummy.corr()

                     Age    Gender  N_Purchases  Purchase_VALUE  Time_Spent  \
Age             1.000000  0.001892    -0.005580        0.006416   -0.001506   
Gender          0.001892  1.000000     0.001474       -0.002496    0.001595   
N_Purchases    -0.005580  0.001474     1.000000       -0.219670    0.001604   
Purchase_VALUE  0.006416 -0.002496    -0.219670        1.000000   -0.003069   
Time_Spent     -0.001506  0.001595     0.001604       -0.003069    1.000000   
Newsletter      0.003068 -0.001282    -0.001593        0.001272    0.003764   
Voucher        -0.001288  0.001538    -0.004484       -0.000461    0.001487   
Pay_Method_0   -0.000930  0.002142    -0.004805       -0.002288   -0.001751   
Pay_Method_1   -0.001691  0.003402     0.000192        0.002956   -0.005369   
Pay_Method_2    0.004828  0.002278     0.003334       -0.000126    0.004341   
Pay_Method_3   -0.002062 -0.008886     0.001810       -0.000676    0.003716   
Browser_0      -0.007451  0.000362    -0.007193     

,Age,Gender,N_Purchases,Time_Spent,Newsletter,Voucher,Pay_Method_0,Pay_Method_1,Pay_Method_2,Pay_Method_3,Browser_0,Browser_1,Browser_2,Browser_3
Age,1.000000,0.001892,-0.005580,-0.001506,0.003068,-0.001288,-0.000930,-0.001691,0.004828,-0.002062,-0.007451,0.004296,0.003853,0.003234
Gender,0.001892,1.000000,0.001474,0.001595,-0.001282,0.001538,0.002142,0.003402,0.002278,-0.008886,0.000362,-0.002603,-0.003393,0.005172
N_Purchases,-0.005580,0.001474,1.000000,0.001604,-0.001593,-0.004484,-0.004805,0.000192,0.003334,0.001810,-0.007193,0.000281,0.002316,0.009061
Time_Spent,-0.001506,0.001595,0.001604,1.000000,0.003764,0.001487,-0.001751,-0.005369,0.004341,0.003716,-0.003711,0.003781,0.001052,0.000122
Newsletter,0.003068,-0.001282,-0.001593,0.003764,1.000000,0.004231,-0.001202,-0.000129,-0.002107,0.003780,-0.000409,-0.003265,-0.000388,0.005082
Voucher,-0.001288,0.001538,-0.004484,0.001487,0.004231,1.000000,0.005051,0.000260,0.000887,-0.007103,0.003517,-0.005718,0.001889,0.000577
Pay_Method_0,-0.000930,0.002142,-0.004805,-0.001751,-0.001202,0.005051,1.000000,-0.418530,-0.340484,-0.308161,-0.000928,0.006370,0.001058,-0.007473
Pay_Method_1,-0.001691,0.003402,0.000192,-0.005369,-0.000129,0.000260,-0.418530,1.000000,-0.349037,-0.315902,-0.001762,-0.000542,0.003624,0.000842
Pay_Method_2,0.004828,0.002278,0.003334,0.004341,-0.002107,0.000887,-0.340484,-0.349037,1.000000,-0.256994,0.006464,-0.011346,0.002849,0.002571
Pay_Method_3,-0.002062,-0.008886,0.001810,0.003716,0.003780,-0.007103,-0.308161,-0.315902,-0.256994,1.000000,-0.003716,0.005283,-0.008495,0.004952


3. **Aufteilen in Trainings, Testdaten und Validierungsdaten**

In [17]:

df_x = df_dummy
df_y = df_reg['Revenue_Total']


df_x_train, df_x_rest, df_y_train, df_y_rest = train_test_split(df_x, df_y, test_size=0.3, random_state=42)
df_x_test, df_x_validation, df_y_test, df_y_validation = train_test_split(df_x_rest, df_y_rest, test_size=0.5, random_state=42)




4. **Modelle mit dem Trainingsdatensatz berechnen** + 5. **Modellauswahl mittels der Validierungsdaten. Metrik: MQA**

 
Wir nehmen für die polynomiele Regression dem Grad 1. Somit haben wir zwar ein eher unflexibles Modell, jedoch können wir dadurch eine kleine Varianz erwarten. Was noch zu bemerken ist, ist das der Validierungsfehler sehr hoch ist, wenn man diesen mit dem durchschnittlichen Revenue_TOTAL von 27.73 vergleicht.

In [18]:
for degree in range(1, 5):
    poly = PolynomialFeatures(degree=degree)
    
    df_x_train_poly = poly.fit_transform(df_x_train)
    model = LinearRegression()

    
    model.fit(df_x_train_poly, df_y_train)

    df_x_validation_poly = poly.transform(df_x_validation) # Validierungsdaten transformieren
    y_pred_validation = model.predict(df_x_validation_poly) # Vorhersage erstellen (Validierungsdaten)
    mse_validation = mean_squared_error(df_y_validation, y_pred_validation)
    print("Grad: "+str(degree)+" Validation Error: "+str(mse_validation))




Grad: 1 Validation Error: 224.07202030371255


Grad: 2 Validation Error: 224.31831448215377
Grad: 3 Validation Error: 224.92342883103103
Grad: 4 Validation Error: 227.3738073618011


6. **Berechnung des Testfehlers. Metrik: MQA**



In [19]:
poly = PolynomialFeatures(degree=1)
df_x_train_poly = poly.fit_transform(df_x_train) # Daten transformieren

model = LinearRegression() 
model.fit(df_x_train_poly, df_y_train) # Model erstellen
df_x_test_poly = poly.transform(df_x_test) # Testdaten transformieren
y_pred_train = model.predict(df_x_train_poly)
y_pred_test = model.predict(df_x_test_poly) # Vorhersage erstellen (Testdaten)
mse_train = mean_squared_error(df_y_train, y_pred_train)
mse_test = mean_squared_error(df_y_test, y_pred_test)
print("Grad: " + str(1) + " Training Error: " + str(mse_train))
print("Grad: "+str(1)+" Test Error: "+str(mse_test))


Grad: 1 Training Error: 222.53586403396156
Grad: 1 Test Error: 225.6920692912952


7. **Anpassung des Modells**
 
Der sehr hohe Trainings und Testfehler zeigt, dass wir einen hohen Bias und keine kleine Varianz haben. Die erste Intuition ist, dass wir ein flexibleres Modell brauchen, jedoch haben wir bereits ein Regressionen mit einem höheren Grad getestet, welche sogar schlechter ausgefallen sind. Dies ist der Grund weshalb wir nun Regressionsbäume testen.
 




8. **Tree**
 
Der hohe Testfehler und sehr niedrieger Trainingsfehler, weist auf eine hohe Varianz und ein niedrigen Bias hin. Um die Varianz in diesem Baum zu reduzieren, kann dieser gestuzt werden. Infolge dessen erwatren wir aber auch einen höheren Bias, aufgrund des Bias-Varianz-Traidoff.

In [20]:
regressor = DecisionTreeRegressor(random_state=42)
regressor.fit(df_x_train, df_y_train)


y_pred_train = regressor.predict(df_x_train)
y_pred_test = regressor.predict(df_x_test)
mse_train = mean_squared_error(df_y_train, y_pred_train)
mse_test = mean_squared_error(df_y_test, y_pred_test)
print("Tree "+str(regressor.get_depth())+" : Training Error: " + str(mse_train))
print("Tree "+str(regressor.get_depth())+" : Test Error: "+str(mse_test))



Tree 54 : Training Error: 0.5364346353431617
Tree 54 : Test Error: 458.7541118654372


9. **Auswahl des Modells**
Leider weist kein Modell einen niedrigen Trainings- oder Validirungsfehler auf. Daher ist keines dieser Modelle geeignet, und es wird keines von ihnen genutzt. Dies ist auch der Grund, weshalb kein weiterer Testfehler errechnet wird.

In [21]:
for max_depth in range(1, 54):
    regressor = DecisionTreeRegressor(max_depth=max_depth,random_state=42)
    regressor.fit(df_x_train, df_y_train)


    y_pred_train = regressor.predict(df_x_train)
    y_pred_validation = regressor.predict(df_x_validation)
    mse_train = mean_squared_error(df_y_train, y_pred_train)
    mse_validation = mean_squared_error(df_y_validation, y_pred_validation)
    print("Tree "+str(regressor.get_depth())+" : Training Error: " + str(mse_train))
    print("Tree "+str(regressor.get_depth())+" : Validation Error: "+str(mse_validation))

Tree 1 : Training Error: 222.5672490728059
Tree 1 : Validation Error: 223.99643288533161
Tree 2 : Training Error: 222.5026897354644
Tree 2 : Validation Error: 224.137159165071
Tree 3 : Training Error: 222.40467181834563
Tree 3 : Validation Error: 224.36882315386043
Tree 4 : Training Error: 222.21154030797013
Tree 4 : Validation Error: 224.74724495150969
Tree 5 : Training Error: 221.8373056911736
Tree 5 : Validation Error: 224.93825253584424
Tree 6 : Training Error: 221.14349525802504
Tree 6 : Validation Error: 225.93821056382117
Tree 7 : Training Error: 219.98606998782506
Tree 7 : Validation Error: 227.06418784782886
Tree 8 : Training Error: 218.27199902042634
Tree 8 : Validation Error: 228.17988389803304
Tree 9 : Training Error: 215.67859726192472
Tree 9 : Validation Error: 232.09111714513406
Tree 10 : Training Error: 212.36332016089747
Tree 10 : Validation Error: 234.98148448723694
Tree 11 : Training Error: 207.64411715333657
Tree 11 : Validation Error: 240.25882920655917
Tree 12 : T

10. **Fazit**

Leider ist es uns nicht gelungen, ein Modell zu erstellen, das den geschätzten Gesamtumsatz liefert. Ein Grund dafür könnte die Datenmenge sein, obwohl wir bereits etwa 60.000 Datensätze haben. Ein weiterer Grund könnte sein, dass es keinen klaren Zusammenhang zwischen dem Gesamtumsatz und den restlichen Merkmalen gibt, weshalb wir kein passendes Modell erstellen können.


# Logistische Regression

In diesem Abschnitt werden wir eine logitische Regression vornehmen. Das Ziel ist es neue Kunden als Groß- oder Klein-Kunde klassifizieren zu können.
Dadurch können Großkunden spezifischer Angebote erhalten. Ein weitere Aspekt wäre die Maximierung des Umsatzes jedes Kunden, wodurch Klein-Kunden im Laufe der Zeit zum Groß-Kunden heranwachsen.

Die Kunden werden als Groß-Kunde klassifiziert wenn ihr durchschnittlicher Umsatz über dem Median liegt und sie mehr als 3 Einkäufe getätigt haben.
Alle anderen Kunden werden als Klein-Kunden klassifiziert.
Hierfür haben wir dem Datensatz eine weitere Spalte als dummy_variable = Großkunde hinzugefügt.  


In [22]:
data_log = df
data_log.head()

df_log = pd.DataFrame(data_log)

df_log = df_log.drop(columns=['Browser', 'Purchase_DATE', 'Customer_id'])

# Korrelationsmatrix aufstellen
print(df_log.corr())

# Korrelation zwischen "Revenue_Total" und "Purchase_Value"
df_log = df_log.drop(columns=['Purchase_VALUE'])

                     Age    Gender  Revenue_Total  N_Purchases  \
Age             1.000000  0.001892       0.000091    -0.005580   
Gender          0.001892  1.000000      -0.001006     0.001474   
Revenue_Total   0.000091 -0.001006       1.000000     0.005794   
N_Purchases    -0.005580  0.001474       0.005794     1.000000   
Purchase_VALUE  0.006416 -0.002496       0.649235    -0.219670   
Pay_Method      0.000750 -0.006452       0.005789     0.004603   
Time_Spent     -0.001506  0.001595      -0.003520     0.001604   
Newsletter      0.003068 -0.001282       0.003578    -0.001593   
Voucher        -0.001288  0.001538       0.000071    -0.004484   

                Purchase_VALUE  Pay_Method  Time_Spent  Newsletter   Voucher  
Age                   0.006416    0.000750   -0.001506    0.003068 -0.001288  
Gender               -0.002496   -0.006452    0.001595   -0.001282  0.001538  
Revenue_Total         0.649235    0.005789   -0.003520    0.003578  0.000071  
N_Purchases          -0

In [23]:
# Hinzufügen einer neuen Spalte (Großkunde 0/1)
median_revenue = df_log['Revenue_Total'].median()
df_log['Großkunde'] = np.where((df_log['Revenue_Total'] > median_revenue) & (df_log['N_Purchases'] > 3), 1,0) 
df_log.head()

,Age,Gender,Revenue_Total,N_Purchases,Pay_Method,Time_Spent,Newsletter,Voucher,Großkunde
0,53,0,45.3,2,1,885,0,0,0
1,18,1,36.2,3,2,656,0,1,0
2,52,1,10.6,1,0,761,1,0,0
3,29,0,54.1,5,1,906,1,0,1
4,21,1,56.9,1,1,605,1,0,0


Die Daten sind nun bereinigt und bereit für die logistische Regression.
Wir haben Daten mit kaum, bis gar keinen Einfluss auf unsere Output Variable Großkunde entfernt.
Dies geschah über die Korellationsmatrix.
Außerdem haben wir die dummy_variable dem Datensatz hinzugefügt.

In [24]:

# Aufteilen Input- und Output-Variablen

inputvar = df_log.drop(columns=['Großkunde'])
outputvar = df_log['Großkunde']

# Aufteilung in Trainings- und Testdaten

inputvar_train, inputvar_test, outputvar_train, outputvar_test = train_test_split(inputvar, outputvar, test_size=0.1, random_state=42)

logreg = LogisticRegression(max_iter=1000)
logreg.fit(inputvar_train, outputvar_train)

output_prediction = logreg.predict(inputvar_test)
genauigkeit = accuracy_score(outputvar_test, output_prediction)
print(f'Genauigkeit: {genauigkeit:.2f}')

confusion = confusion_matrix(outputvar_test, output_prediction)
print(confusion)

Genauigkeit: 0.88
[[4373  348]
 [ 427 1432]]


Das Modell ist mit 88% Genauigkeit effizient.
Es ist jedoch zu beachten, dass die dummy_Variable auf Grundlage unserer Definition berechnet wurde. 
Somit ist der Zusammenhang zwischen den Input-Variablen und der Output-Variable klar. 
Dennoch soll das Modell bei zukünftig großen Datensetzen eine valide Klassifikation vornehmen. 